In [1]:
!pip install transformers==4.28.0

!git clone https://github.com/ZIZUN/korean-malicious-comments-dataset.git

import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

#GPU 설정

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:', device)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.6 MB/s eta 0:00:00
Cloning into 'korean-malicious-comments-dataset'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 30 (delta 8), reused 2 (delta 2), pack-reused 21
Unpacking objects: 100% (30/30), 525.17 KiB | 2.48 MiB/s, done.
device: cuda:0


In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/댓글들/Dataset.csv',sep='\t')
df.head()

,content,lable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   content  10000 non-null  object 
 1   lable    9975 non-null   float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB


In [ ]:
null_idx = df[df.lable.isnull()].index
df.loc[null_idx,"content"]

1602    응애 응애 엄마 저 맘에 안들죠? ........아들 ?? " 너 내가 우스워 보이...
1654           토니스타크 평소 "아이엠그루트"라는 유행어를 부러워했다는게 학계의 정설\t1
1992    "13일 현대차에 따르면 올 들어 국내 소비자들의 수입차 구매의향률이 3년 만에 하...
2920                 에이프릴이 한마디 합니다 "예쁜게 죄" 구하라님 "무기징역"\t1
3720          답글 글씨체를 봐라 저게 애새끼가 쓴거냐?"빨갱이새끼가 쓴거지 ㅁㅈㅎㅉㅉ\t0
3807    알겠다이기ㅋㅋ 딱 채찍쳐맞는거 좋아하는 한국식 마인드네. 노예마인드. 조금만 성공한...
3908           이래서 스스로 걸리거든 "죄인들이"~ㅎㅎㅎ 재미보고 털리고 그치~~~?\t0
4241    아버지는 내재된 악마들을 다룰 정신적 힘을 가지고 있지 않았다." 이 말한마디가 사...
4283    댓글 중 "선동 당해서 촞불든 개돼지 홍어들도 단죄를 받아야 할 공범자들이다"에10...
5000    스파이 제안받고 살해 안당하는 법1. 처음에 스파이 제안을 받았을때 "중국을 위해서...
5521    "국방부 "까지 ㅡㄱ ㅐ 엿같은 ㅈ ㅣ랄주댕이...좌빨에서 ㅡ인민군대로 ㅡ가려는건가...
5866    쌩뚱맞게 60대최반엌 치매라니 그것도 곱게 사는 사모님이- -" 알콜중독도 아니고 ...
6477    페미메퇘지쿵쾅년인 메갈페미들은 니들이 좋아하는 싫어요 ㄱㄱ제발부탁해~~"일반 여성"...
6538    아니 ㅆㅂ 그런 "카더라"가 넘쳐난다고 그거에 대해서 혹시 댓글게이는 뭔가 아는거 ...
6771    저 때 투니버스에서 코요태 짧게 인터뷰 했었는데 김종민이 "노래는 뭐 신지가 다 하...
6932               개 족 가튼 국방부의 "휴기연장콜센터"발족을 축하한다 ㅆ ㅂ..\t0
7199    민족적 자존심과 애국심을 갖고 국산품 이용합시다 . . . "겸손"한 마음으로 재산...
7252    아나운서는 

In [ ]:
df.loc[null_idx,"lable"] = df.loc[null_idx, "content"].apply(lambda x: x[-1])
df.loc[null_idx,"content"] = df.loc[null_idx, "content"].apply(lambda x: x[-2])

In [ ]:
df =df.astype({"lable":"int"})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  10000 non-null  object
 1   lable    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [ ]:
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

In [ ]:
#데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(test_data)))

#중복 데이터 제거
train_data.drop_duplicates(subset=["content"],inplace=True)
test_data.drop_duplicates(subset=["content"],inplace=True)

#데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))



중복 제거 전 학습 데이터셋 : 8000
중복 제거 전 테스트 데이터셋 : 2000
중복 제거 후 학습 데이터셋 : 7972
중복 제거 후 테스트 데이터셋 : 1997


In [ ]:
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
tokenized_train_sentences = tokenizer(list(train_data["content"]),
                                     return_tensors="pt",
                                     max_length = 128,
                                     padding = True,
                                     truncation = True,
                                     add_special_tokens = True,)

In [ ]:
print(tokenized_train_sentences[0])
print(tokenized_train_sentences[0].tokens)
print(tokenized_train_sentences[0].ids)
print(tokenized_train_sentences[0].attention_mask)

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '국방부', '~', '~', '전화로', '휴가', '##연장', '##을', '한', '병사', '##들', '몇이나', '되는지', '공개해라', '~', '어느', '훌륭한', '집안', '##의', '자제', '##분들', '##인지도', '같이', '공개해라', '~', '~', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [ ]:
tokenized_test_sentences = tokenizer(list(test_data["content"]),
                                     return_tensors="pt",
                                     max_length = 128,
                                     padding = True,
                                     truncation = True,
                                     add_special_tokens = True,)

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels):
    self.encodings = encodings
    self.labels = labels
  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

In [ ]:
train_label = train_data["lable"].values
test_label = test_data["lable"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [ ]:
noncontext_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
noncontext_model.to(device)   #device는 gpu로 하는거임

Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'c

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
training_args = TrainingArguments(
output_dir='./',
num_train_epochs=10,
per_device_train_batch_size=8,
per_device_eval_batch_size=64, #배치사이즈 eval train 맞추기
logging_dir='./logs',
logging_steps=500,
save_total_limit=2,
)

In [ ]:
def compute_metrics (pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, fl, _ = precision_recall_fscore_support(labels, preds, average='binary')
  acc = accuracy_score (labels, preds)
  return {
    'accuracy': acc,
    'fl': fl,
    'precision': precision,
    'recall': recall
}

In [ ]:
trainer =Trainer(
model=noncontext_model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=test_dataset,
compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-20-7e7a2c866303>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}


Step,Training Loss
500,0.362600
1000,0.311200
1500,0.189000
2000,0.177000
2500,0.090900
3000,0.089900
3500,0.042300
4000,0.052700
4500,0.025500
5000,0.027800


<ipython-input-20-7e7a2c866303>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
<ipython-input-20-7e7a2c866303>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
<ipython-input-20-7e7a2c866303>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
<ipython-input-20-7e7a2c866303>:6: UserWarning: To copy construct from a ten

TrainOutput(global_step=9970, training_loss=0.07563144445658447, metrics={'train_runtime': 952.8633, 'train_samples_per_second': 83.664, 'train_steps_per_second': 10.463, 'total_flos': 5243803333324800.0, 'train_loss': 0.07563144445658447, 'epoch': 10.0})

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)

<ipython-input-20-7e7a2c866303>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}


{'eval_loss': 0.6814855337142944,
 'eval_accuracy': 0.9183775663495243,
 'eval_fl': 0.9178841309823678,
 'eval_precision': 0.911,
 'eval_recall': 0.9248730964467005,
 'eval_runtime': 4.1201,
 'eval_samples_per_second': 484.702,
 'eval_steps_per_second': 7.767,
 'epoch': 10.0}

In [ ]:
from sklearn.metrics import confusion_matrix
# imports
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

y_pred = []
y_true = []

# iterate over test data
for inputs, labels in df2:
        output = net(inputs) # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

# constant for classes
classes = (0,1)

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')

ValueError: ignored

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/댓글들/통합 문서3.csv')
df2.head()

,comment,label
0,순식간에 문따고 들어가 먹방하는 웅이 폼클렌징 미쳤다;;,1
1,라면보다 사람을 더 매콤하게 만드셨네 ㄷㄷ,1
2,나도 웅이한테 집착당하고 싶다 ㅠㅠ,1
3,지인들의 폭로가 넘치고있습니다 지인분들은 제보 더 해주십쇼,1
4,진짜 궁금한게 커튼 뒤에는 왜 숨어 있던거야?,1


In [ ]:
idx = df2[df2['label'] == 0].index
df2.drop(idx , inplace=True)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  1275 non-null   object
 1   label    1275 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.0+ KB


In [ ]:
null_idx = df2[df2.label.isnull()].index
df2.loc[null_idx,"comment"]

Series([], Name: comment, dtype: object)

In [ ]:
df2.loc[null_idx,"label"] = df2.loc[null_idx, "comment"].apply(lambda x: x[-1])
df2.loc[null_idx,"comment"] = df2.loc[null_idx, "comment"].apply(lambda x: x[-2])

In [ ]:
df2 =df2.astype({"label":"int"})
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  1275 non-null   object
 1   label    1275 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.0+ KB


In [ ]:
train_data = df2.sample(frac=0.8, random_state=42)
test_data = df2.drop(train_data.index)

In [ ]:
#데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(test_data)))

#중복 데이터 제거
train_data.drop_duplicates(subset=["comment"],inplace=True)
test_data.drop_duplicates(subset=["comment"],inplace=True)

#데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))



중복 제거 전 학습 데이터셋 : 1324
중복 제거 전 테스트 데이터셋 : 331
중복 제거 후 학습 데이터셋 : 1314
중복 제거 후 테스트 데이터셋 : 331


In [ ]:
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
tokenized_train_sentences = tokenizer(list(train_data["comment"]),
                                     return_tensors="pt",
                                     max_length = 128,
                                     padding = True,
                                     truncation = True,
                                     add_special_tokens = True,)

In [ ]:
print(tokenized_train_sentences[0])
print(tokenized_train_sentences[0].tokens)
print(tokenized_train_sentences[0].ids)
print(tokenized_train_sentences[0].attention_mask)

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '뭐', '아무', '표명', '##도', '##없네', 'ㅋㅋ', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [ ]:
tokenized_test_sentences = tokenizer(list(test_data["comment"]),
                                     return_tensors="pt",
                                     max_length = 128,
                                     padding = True,
                                     truncation = True,
                                     add_special_tokens = True,)

In [ ]:
train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [ ]:
model_context = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model_context.to(device)

Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'c

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
training_args = TrainingArguments(
output_dir='./',
num_train_epochs=10,
per_device_train_batch_size=8,
per_device_eval_batch_size=64,
logging_dir='./logs',
logging_steps=500,
save_total_limit=2,
)

In [ ]:
trainer =Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=test_dataset,
compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-15-7e7a2c866303>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}


Step,Training Loss
500,0.550200
1000,0.529200
1500,0.521400


<ipython-input-15-7e7a2c866303>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
<ipython-input-15-7e7a2c866303>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
<ipython-input-15-7e7a2c866303>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}


TrainOutput(global_step=1650, training_loss=0.5324850648822207, metrics={'train_runtime': 154.0251, 'train_samples_per_second': 85.311, 'train_steps_per_second': 10.713, 'total_flos': 864319816857600.0, 'train_loss': 0.5324850648822207, 'epoch': 10.0})

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)

<ipython-input-15-7e7a2c866303>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}


{'eval_loss': 0.6079455614089966,
 'eval_accuracy': 0.7220543806646526,
 'eval_fl': 0.8385964912280701,
 'eval_precision': 0.7220543806646526,
 'eval_recall': 1.0,
 'eval_runtime': 0.6845,
 'eval_samples_per_second': 483.589,
 'eval_steps_per_second': 8.766,
 'epoch': 10.0}

In [ ]:
def sentence_predict(sent):
  model.eval()

  tokenized_sent = tokenizer(
      sent,
      return_tensors="pt",
      truncation = True,
      max_length = 128

  )

  tokenized_sent.to(device)

  with torch.no_grad():
    outputs = model(
        input_ids = tokenized_sent["inputs_ids"],
        attention_mask=tokenized_sent["attention_mask"],
        token_type_ids=tokenized_sent["token_type_ids"]
    )

  logits = outputs[0]
  logits = logits.detach().cpu()
  result = logits.argmax(-1)
  if result == 0:
    result = "악성댓글입니다."
  elif result == 1:
    result = "정상댓글입니다."
  return result


while True:
  sentence = input("댓글을 입력해보세요: ")
  if sentence == 0:
    break
  print(sentence_predict(sentence))
  print("\n")







댓글을 입력해보세요: 0


KeyError: ignored

In [ ]:
def example(a):
  sentence = input("안녕 !")
  if sentence == 1:
  print()